## Sistemas de recomendação

Material baseado no post de Abhinav Ajitsaria sobre sistemas de recomendação, disponível em https://madewithml.com/topics/recommendation-systems/

Sistemas de recomendação utilizam várias técnicas, entre elas a filtragem colaborativa. A filtragem colaborativa é uma técnica que pode filtrar itens que um usuário possa gostar com base nas reações de usuários semelhantes. Ele funciona pesquisando um grande grupo de pessoas e encontrando um conjunto menor de usuários com gostos semelhantes a um usuário específico. Ele analisa os itens de que gostam e os combina para criar uma lista classificada de sugestões.

Para experimentar algoritmos de recomendação, você precisará de dados que contenham um conjunto de itens e um conjunto de usuários que reagiram a alguns dos itens. A reação pode ser explícita (avaliação em uma escala de 1 a 5, gosta ou não gosta) ou implícita (ver um item, adicioná-lo a uma lista de desejos, o tempo gasto em um artigo). Ao trabalhar com esses dados, você os verá principalmente na forma de uma matriz que consiste nas reações dadas por um conjunto de usuários a alguns itens de um conjunto de itens.

Existem muitas maneiras de decidir quais usuários são semelhantes e combinar suas escolhas para criar uma lista de recomendações. Veremos a seguir, algumas formas de fazer isso com Python.

### Como identificar usuários com gostos parecidos na base

Para entender o conceito de similaridade, vamos criar um conjunto de dados simples primeiro. Os dados incluem quatro usuários A, B, C e D, que avaliaram dois filmes. As classificações são armazenadas em listas e cada lista contém dois números que indicam a classificação de cada filme.

In [1]:
from scipy import spatial
import matplotlib.pyplot as plt
import pandas as pd

a = [1, 2]
b = [2, 4]
c = [2.5, 4]
d = [4.5, 5]

labels = 'abcd'

dataset = pd.DataFrame([a,b,c,d])

plt.style.use("ggplot")
fig, ax = plt.subplots()
ax.scatter(dataset[0], dataset[1])

ax.set_xlim(0, 6)
ax.set_ylim(0, 6)

for i, item in enumerate([a,b,c,d]):
    ax.annotate(labels[i], (item[0]+0.06, item[1]+0.06))

In [2]:
print("C para A:", spatial.distance.euclidean(c, a))
print("C para B:", spatial.distance.euclidean(c, b))
print("C para D:", spatial.distance.euclidean(c, d))

C para A: 2.5
C para B: 0.5
C para D: 2.23606797749979


Olhar para a distância entre os pontos parece ser uma boa maneira de estimar a similaridade. Usar a distância euclidiana para calcular a distância entre as classificações de A, B e D em relação a C nos mostra que, **em termos de distância, as classificações de C estão mais próximas das de B**. Mas considerando A e D, de quem C está mais próximo? Pode-se dizer que C está mais perto de D em termos de distância. 

No entanto, parece que as escolhas de C se alinham com as de A mais do que D, porque tanto A quanto C gostam do segundo filme quase duas vezes mais do que do primeiro, mas D gosta de ambos os filmes igualmente.

Para contornar essa dificuldade, você pode usar o **cosseno do ângulo para encontrar a semelhança entre dois usuários**. Para calcular a similaridade usando o ângulo, você precisa de uma função que retorne uma similaridade maior ou distância menor para um ângulo inferior e uma similaridade menor ou distância maior para um ângulo maior. O cosseno de um ângulo é uma função que diminui de 1 para -1 conforme o ângulo aumenta de 0 para 180. Quanto maior o ângulo, menor será o cosseno e, portanto, menor será a similaridade dos usuários. Você também pode inverter o valor do cosseno do ângulo para obter a distância do cosseno entre os usuários, subtraindo-o de 1.

In [3]:
print("C para A:", spatial.distance.cosine(c, a))
print("C para B:", spatial.distance.cosine(c, b))
print("C para D:", spatial.distance.cosine(c, d))

C para A: 0.004504527406047898
C para B: 0.004504527406047898
C para D: 0.015137225946083022


A similaridade agora aponta que C tem gostos similares com A e B, mas mais diferente dos gostos de D.

### Modelo de sistemas de recomendação

Para indicar itens a um determinado usuário, é necessário "predizer" qual seria a nota dada por um determinado usuário. Para isso, são consideradas as médias de notas para aquele item referente aos usuários mais parecidos. Para isso, vamos utilizar uma biblioteca de algoritmos de sistemas de recomendação chamada *surprise*.

In [4]:
!pip install scikit-surprise

Para carregar os dados, a biblioteca surprise oferece alguns métodos. Como os dados precisam ser armazenados de maneira diferente, numa relação de itens por usuário, esses métodos acabam sendo úteis para converter datasets no formato tradicional.

In [5]:
import pandas as pd
from surprise import Dataset
from surprise import Reader

# conjunto anterior, com o usuário E que só avaliou o filme 1
ratings_dict = {
    "item": [1, 2, 1, 2, 1, 2, 1, 2, 1],
    "user": ['A', 'A', 'B', 'B', 'C', 'C', 'D', 'D', 'E'],
    "rating": [1, 2, 2, 4, 2.5, 4, 4.5, 5, 3],
}

df = pd.DataFrame(ratings_dict)
reader = Reader(rating_scale=(1, 5))

# carregando a partir de dataframe do pandas
data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)

Para executar algum algoritmo de sistema de recomendação sobre o conjunto de dados é necessário definir qual método a ser utilizado e a biblioteca surprise traz vários implementados.

In [6]:
from surprise import KNNWithMeans

knn_options = {
    "name": "cosine",
    "user_based": False,  # calcula similaridade entre os itens
}

algo = KNNWithMeans(sim_options=knn_options)

Para experimentar este recomendador, você precisa criar um Trainset a partir de dados. O Trainset é construído usando os mesmos dados, mas contém mais informações sobre os dados, como o número de usuários e itens (n_users, n_items) que são usados pelo algoritmo. Você pode criá-lo usando todos os dados ou parte dos dados. Você também pode dividir os dados em partes, onde alguns dos dados serão usados para treinamento e outros para teste.

In [7]:
trainingSet = data.build_full_trainset()
algo.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


Agora vamos estimar a nota que o usuário E daria para o filme 2.

In [8]:
prediction = algo.predict('E', 2)
prediction.est

4.15

### Utilizando uma base de dados real

In [9]:
from surprise.model_selection import GridSearchCV

data = Dataset.load_builtin("ml-100k")

options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

O retorno do método aponta **0,94 de RMSE** e os melhores parâmetros foram o método msd, suporte mínimo de 3, com a abordagem baseada em itens.

---

## AC2 - Não-supervisionado

A primeira atividade da AC2 para entregar dia 30/08 consiste em avaliar outros 2 algoritmos disponíveis na biblioteca Surprise, mantendo fixo a validação cruzada com 3 partições (cv=3). Deve existir uma explicação minimalista dos métodos e dos parâmetros avaliados. Os parâmetros avaliados devem ser pelo menos 2 para os métodos diferentes. **Essa atividade pode ser feita em grupos de até 4 pessoas.**